In [4]:
!pip install transformers torch gradio PyPDF2

In [5]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import PyPDF2

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch

model_path = "ibm-granite/granite-3.3-2b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map=device,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

tokenizer = AutoTokenizer.from_pretrained(model_path)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

In [7]:
# Generate eco-friendly tips
def tips_generator(problem_keywords):
    prompt = f"""
You are an eco-assistant helping urban residents in India live sustainably.

Problem: {problem_keywords}

Here are 5 practical, diverse, and context-aware eco-friendly tips:

1. Use cloth bags instead of plastic when shopping.
2. Install low-flow faucets to reduce water usage.
3. Compost kitchen waste to enrich soil and reduce landfill.
4. Switch to LED bulbs to save energy.
5. Use public transport or carpool to reduce emissions.

Now generate 5 new tips for the same problem:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(**inputs, max_length=200, do_sample=True, temperature=0.5, top_k=50, top_p=0.9)
    eos_token_id = tokenizer.encode("###")[0]
    eos__token_id=tokenizer.eos_token_id

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()


# Summarize sustainability policies
def policy_summarization(pdf_file, policy_text):
    summary_prompt = ""

    if pdf_file is not None:
        try:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            summary_prompt = f"Summarize the following sustainability policy:\n{text}"
        except Exception as e:
            return f"Error reading PDF: {e}"

    elif policy_text.strip() != "":
        summary_prompt = f"Summarize the following policy document:\n{policy_text}"
    else:
        return "⚠️ Please upload a file or enter policy text."

    inputs = tokenizer(summary_prompt, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=300)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()

In [8]:
with gr.Blocks() as app:
    gr.Markdown("## 🌆 Sustainable Smart City Assistance - Eco Assistant & Policy Analyzer")

    with gr.Tabs():
        # Tab 1: Eco Tips
        with gr.Tab("Eco Tips Generator"):
            with gr.Row():
                keywords_input = gr.Textbox(
                    label="Environmental Problem/Keywords",
                    placeholder="e.g., plastic, solar, waste management, energy saving...",
                    lines=3
                )
            tips_output = gr.Textbox(label="🌿 Sustainable Living Tips", lines=10)
            generate_btn = gr.Button("Generate Eco Tips")
            generate_btn.click(tips_generator, inputs=keywords_input, outputs=tips_output)

        # Tab 2: Policy Summarization
        with gr.Tab("Policy Summarization"):
            with gr.Row():
                pdf_upload = gr.File(label="📄 Upload Policy PDF", file_types=[".pdf"])
                policy_text_input = gr.Textbox(
                    label="Or Paste Policy Text",
                    placeholder="Paste sustainability policy text here...",
                    lines=6
                )
            summary_output = gr.Textbox(label="📌 Policy Summary & Key Points", lines=12)
            summarize_btn = gr.Button("Summarize Policy")

            summarize_btn.click(
                policy_summarization,
                inputs=[pdf_upload, policy_text_input],
                outputs=summary_output
            )

# Launch app with shareable link
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7e788f7ce947419602.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
